In [22]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel
import json
import requests
from rapidfuzz import process, fuzz
import pickle

In [53]:
credits=pd.read_csv('tmdb_5000_credits.csv')
movies=pd.read_csv('tmdb_5000_movies.csv')

In [54]:
credits_rename=credits.rename(index=str,columns={'movie_id':"id"})
data=movies.merge(credits_rename,on='id')

In [55]:
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [57]:
data=data.fillna(" ")

In [58]:
data['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [59]:
data['production_companies']

0       [{"name": "Ingenious Film Partners", "id": 289...
1       [{"name": "Walt Disney Pictures", "id": 2}, {"...
2       [{"name": "Columbia Pictures", "id": 5}, {"nam...
3       [{"name": "Legendary Pictures", "id": 923}, {"...
4             [{"name": "Walt Disney Pictures", "id": 2}]
                              ...                        
4798             [{"name": "Columbia Pictures", "id": 5}]
4799                                                   []
4800    [{"name": "Front Street Pictures", "id": 3958}...
4801                                                   []
4802    [{"name": "rusty bear entertainment", "id": 87...
Name: production_companies, Length: 4803, dtype: object

In [60]:
for i in range(0,4803):
    try:
        
        data['production_companies'][i]=data['production_companies'][i][1:-1].split('"')[3]
    except:
        pass

for j in range(0,4803):
    gene=''
    try:
        for i in range(0,10):
            gene=gene + " " +json.loads(data['genres'][j])[i]['name']
    except:
        pass
    try:
        
        data['genres'][j]=gene
    except:
        pass

c:\users\acer\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\acer\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [61]:
data['new']=data['overview']+"  "+data['production_companies']+" "+data['tagline']

In [62]:
tfv=TfidfVectorizer(min_df=3,max_features=None,strip_accents='unicode',
                    analyzer='word',token_pattern=r'\w{1,}',stop_words='english')
data['new']=data['new'].fillna('')
tvf_matrice=tfv.fit_transform(data['new'])
sig=sigmoid_kernel(tvf_matrice,tvf_matrice)
indices=pd.Series(data.index,index=data['original_title']).drop_duplicates()

In [94]:
def give_rec(title,sig=sig):
    idx=process.extractOne(title, data['original_title'])[2]
    sig_score=list(enumerate(sig[idx]))
    sig_score=sorted(sig_score,key=lambda X:X[1],reverse=True)
    sig_score=sig_score[1:20]
    movie_indices=[i[0] for i in sig_score]
    return data['original_title'].iloc[movie_indices]

In [95]:
give_rec('iron man')

31                                           Iron Man 3
79                                           Iron Man 2
7                               Avengers: Age of Ultron
182                                             Ant-Man
94                              Guardians of the Galaxy
1868                                 Cradle 2 the Grave
618                                         Mystery Men
119                                       Batman Begins
1404                                     Street Fighter
169                  Captain America: The First Avenger
538                                             Hostage
1290                           Baahubali: The Beginning
2633                          The Clan of the Cave Bear
4574                                           Roadside
3720    R.L. Stine's Monsterville: The Cabinet of Souls
4407                                               Hits
401                                  Enemy at the Gates
735                                          The

In [75]:
len(data['id'])

4803

In [66]:
data.iloc[31].id

68721

collecting movies images

In [67]:
temp=requests.get(f'https://api.themoviedb.org/3/movie/{68721}?api_key=49a550a06e7b03b2237a7f05976ca9e8').json()

In [71]:
poster=[]
i=0
for id in data['id']:
    print(f'iteration={i} id={id}')
    temp=requests.get(f'https://api.themoviedb.org/3/movie/{id}?api_key=49a550a06e7b03b2237a7f05976ca9e8').json()
    try:
        poster.append(temp['belongs_to_collection']['poster_path'])
    except:
        try:
            poster.append(temp['backdrop_path'])
            print(temp['backdrop_path'])
        except:
            poster.append(" ")
    i=i+1

iteration=0 id=19995
iteration=1 id=285
iteration=2 id=206647
iteration=3 id=49026
iteration=4 id=49529
/ubNevniKd5NYo60DqYIQ29T0uyB.jpg
iteration=5 id=559
iteration=6 id=38757
/i25Qs826elaSSBrAtJMVO3mfuZB.jpg
iteration=7 id=99861
iteration=8 id=767
iteration=9 id=209112
iteration=10 id=1452
iteration=11 id=10764
iteration=12 id=58
iteration=13 id=57201
/esxoiZoDQydeyEX1tJcSbuigI3a.jpg
iteration=14 id=49521
iteration=15 id=2454
iteration=16 id=24428
iteration=17 id=1865
iteration=18 id=41154
iteration=19 id=122917
iteration=20 id=1930
iteration=21 id=20662
/h7AKqgIh86HDgJIzENeR2ng1Knn.jpg
iteration=22 id=57158
iteration=23 id=2268
/szAuHzSVFzamjCkkulopJZQVNYd.jpg
iteration=24 id=254
/b9XoSklkXIDIOQh6otfs0l8YLCH.jpg
iteration=25 id=597
/6VmFqApQRyZZzmiGOQq2C92jyvH.jpg
iteration=26 id=271110
iteration=27 id=44833
/hgFIvyAAPvQ72jGOkAry6ZtDQo3.jpg
iteration=28 id=135397
iteration=29 id=37724
iteration=30 id=558
iteration=31 id=68721
iteration=32 id=12155
iteration=33 id=36668
iteration=34 

/yvFtIWMvRS8PUbJOGpkSZk07CB3.jpg
iteration=229 id=1895
iteration=230 id=1894
iteration=231 id=585
iteration=232 id=76170
iteration=233 id=1893
iteration=234 id=49519
iteration=235 id=2395
iteration=236 id=12100
/xr60oCXkfFIqMRJ4YxtMiD53zBy.jpg
iteration=237 id=290595
iteration=238 id=98566
iteration=239 id=49047
/a2n6bKD7qhCPCAEALgsAhWOAQcc.jpg
iteration=240 id=9619
/mnXOfgasuAldIwmTRB0wQKJSsz4.jpg
iteration=241 id=308531
iteration=242 id=166424
/gzhVcfC5j0sTnS4WJzBrCS0Vkr0.jpg
iteration=243 id=1593
iteration=244 id=254128
/mMoG4nPSDupXIXOwVvpexZY2W0N.jpg
iteration=245 id=714
iteration=246 id=2024
/c9xSFZQtC4c16Vn5CMOaB1pGk9J.jpg
iteration=247 id=163
iteration=248 id=787
/iFaWUETEE2umP1WY0LUAmjSpzm7.jpg
iteration=249 id=262500
iteration=250 id=2567
/wNBLjvDpvTS3wyOZCecf3Jas7la.jpg
iteration=251 id=38745
/1EnX8qIJUYv6mG0Asub6Om8158z.jpg
iteration=252 id=40805
/aT301Z8pzjzmCmhjeoLMNWkg8sH.jpg
iteration=253 id=53182
iteration=254 id=41513
iteration=255 id=13700
/rQKlcMY8i3elENXEHlkAbQr3xi

iteration=429 id=7518
/kmkmZIYAK1m2Q9ez03NJH6y7oNB.jpg
iteration=430 id=11544
iteration=431 id=9986
/xyyu51eOARP9gslKJ0AvyaJkHzA.jpg
iteration=432 id=8656
/w53FK3MoMLavk1sxErUiGMnLpf2.jpg
iteration=433 id=146216
iteration=434 id=9291
/li5i1WXMySEqLMUrc60UTeI99gX.jpg
iteration=435 id=55301
iteration=436 id=109418
iteration=437 id=11665
/yWv0MFof8HucEowvXSUaAh25EaO.jpg
iteration=438 id=6964
/bgOoq2T5ABiLk1Zo9NBjWZdA1vM.jpg
iteration=439 id=11324
/8xt8AMb1OKC63AdhNSaYBWxB4Iq.jpg
iteration=440 id=12193
/nOwpM0Bf15Z3h4p93BciERdEhPW.jpg
iteration=441 id=9928
/8p4gCdDT55PMY3UMDvo6ybzMrai.jpg
iteration=442 id=754
/tsSWHgaFlzzkp3o976juBZbmcs2.jpg
iteration=443 id=10202
/lrQtbANU5t61tNqZASVmQqgXasb.jpg
iteration=444 id=4147
/1xUQlIMFevh7krDDTOkgBhdS0Wy.jpg
iteration=445 id=50546
/sRJXffIo0tY2acAUlIThoopsxia.jpg
iteration=446 id=1701
/2NjpyW4PCWjtuA0bS4fP3d2aEHl.jpg
iteration=447 id=13027
/hPaA2EAKOO6zp55yus0dAoCvbcc.jpg
iteration=448 id=2289
/8H2OE6o14Agg3pIv2Kk6rZBVXsj.jpg
iteration=449 id=2050

iteration=609 id=107846
iteration=610 id=188207
/9LctHvfUwzcT7RzwhRp8SpW5K00.jpg
iteration=611 id=4614
/pXECs3KNnsMBYHK4uA19huGQeSE.jpg
iteration=612 id=24021
iteration=613 id=11371
/tCBcuOw60wjzLvg9oJdZKlBn1nG.jpg
iteration=614 id=20352
iteration=615 id=11517
/oWfITuo0TdJYVRalEH42fuNEN2E.jpg
iteration=616 id=214756
iteration=617 id=26428
/hmdcBI1ScKCt1nGvJGhLTYUHnnH.jpg
iteration=618 id=9824
/hhw6DMJ4AQ5ifHyBENEwCgsLthI.jpg
iteration=619 id=48988
/v0zSbmZ1g5X4C93FZ3o24fG2mB1.jpg
iteration=620 id=9008
/c2fKZrEaMv4aNyEs2W9juh9OGiT.jpg
iteration=621 id=300673
/6XJ5iKhwFb0rY3P8PTjfsgdSZ9u.jpg
iteration=622 id=12113
/iViUI5bTnYkgQ9ty1CXQ8J4obby.jpg
iteration=623 id=38778
/2bNvoxVPyfiuNh7vmRvwePY5Paf.jpg
iteration=624 id=72331
/yUrJTqCUKvfu0FWXm4xFJyZwGMG.jpg
iteration=625 id=1844
/dATdQRgxEcXzLewWNGrjxdNg8wP.jpg
iteration=626 id=846
iteration=627 id=9703
/tgaMo8q1HpGTbtpmJm8TYKTpOr4.jpg
iteration=628 id=857
/hjQp148VjWF4KjzhsD90OCMr11h.jpg
iteration=629 id=136797
/syVAWAnY6fTBDDFntHsM8vFjK

/zZ0rWhPEGQkY24q6MCgI7HZBNwe.jpg
iteration=784 id=2312
iteration=785 id=9839
/zKM4vKNPX3JQamUiJjYZpox5Igb.jpg
iteration=786 id=381902
iteration=787 id=13922
/oEe615shWoM5Wf78ddORdo6WDWp.jpg
iteration=788 id=293660
iteration=789 id=9713
/e45CObYN29s1ZXL9fT0IuRN7aB2.jpg
iteration=790 id=190859
/8y8Ey2XssEl8EYCgnaMaV0AQAUJ.jpg
iteration=791 id=257445
iteration=792 id=9007
/6M0UJRHU4O0lyi0MMOq7TywNTmR.jpg
iteration=793 id=889
iteration=794 id=1370
iteration=795 id=4942
/4ZwVpZrTkTegVEQDUMyYwvT8HL7.jpg
iteration=796 id=347969
/1yAtke4amRvHlzj4nzDtwOC2hJl.jpg
iteration=797 id=24438
/zNA2zYGn2nfoGOHTQXG9l0bLBR8.jpg
iteration=798 id=116741
/kms4PhWfP1Q2jidPLOf26qm0paJ.jpg
iteration=799 id=35791
iteration=800 id=72431
/k0q6OmVjVsMIocC3ovjh1Vzgx7J.jpg
iteration=801 id=1813
/i9lR2jLV0dHVfVs1u6QqqbQAE9I.jpg
iteration=802 id=87428
/ukYpvqpB2QiH0A1NzvV5ylsiUgA.jpg
iteration=803 id=8840
iteration=804 id=10589
/oHF8XZPuXH1J2S9FyagOdeCwz14.jpg
iteration=805 id=71676
iteration=806 id=1722
/zeUeJLEEdfyGH

...

/q4MWby5NSnAImy7sqOPEwYH1YpU.jpg
iteration=3975 id=248
/d96bhq2r9GxoRoAv6COZzuDXtlO.jpg
iteration=3976 id=325173
/93ZSd7uYaJAGqQGutR3v3eynMbl.jpg
iteration=3977 id=55831
/yRDLX79lgo2DiiQuNYfNTYVyVBm.jpg
iteration=3978 id=351043
/mcMxID0P4Rpb5Nt79fZ4tgOvq8e.jpg
iteration=3979 id=43942
None
iteration=3980 id=10226
/ecn4qv1SzQaZ8jMyPPs307UqKxa.jpg
iteration=3981 id=66942
/qKiARMqwgLw8SU14oC1qOmyWUWP.jpg
iteration=3982 id=356483
None
iteration=3983 id=10476
/7B05PWjXIW7uiO3x5ZpmuTQJsIO.jpg
iteration=3984 id=239
/iSbhQqeDnkvFuP7QWwy0qeX0LYy.jpg
iteration=3985 id=10281
iteration=3986 id=630
iteration=3987 id=3034
/14AH4fovT9lj5wFXiXsgBcEKHFH.jpg
iteration=3988 id=13025
/vjedSpn9vaN3Xxzr7MxW9RFGW50.jpg
iteration=3989 id=21461
iteration=3990 id=55306
None
iteration=3991 id=17264
iteration=3992 id=346081
None
iteration=3993 id=16016
/k8ALJxeyPPfwJiLGCGWPWAXDcVe.jpg
iteration=3994 id=15875
/tPssChaKUFIfE54jbWTGpDu4d4K.jpg
iteration=3995 id=5900
None
iteration=3996 id=43306
/cez9he2zb0Jv2AdLhdxcp

/AbPxK4IMO9eNUgJYeSOfOjHxfkz.jpg
iteration=4148 id=198062
/3zaABMNlqu8Mibo7W4RYGqKSyoE.jpg
iteration=4149 id=188652
/xqxlCt4taV1gqCISRoVCRI3Af08.jpg
iteration=4150 id=174311
/5dVuekvxRamQcLJEcJXmGDjxaWk.jpg
iteration=4151 id=12602
/aWqx6rASnrDMvd3jG3gDFwALnyJ.jpg
iteration=4152 id=153795
/xCt7Ff7QgRvL6yF1Q1oJgK7yPlD.jpg
iteration=4153 id=17768
/mtGgCabyz2tkxes2Gq9bJSZsrCk.jpg
iteration=4154 id=13516
/iYcYlFINz3vCFRlYl87yjeq4IhJ.jpg
iteration=4155 id=98549
/st9JYEdSCaRfqzkctfPRUJDSUPJ.jpg
iteration=4156 id=312793
None
iteration=4157 id=309919
/xpkreMLtHfudy6aNr7ZfIFsXduu.jpg
iteration=4158 id=299553
/gh5pdz3aoMpPuOnafOq3N1PWlPx.jpg
iteration=4159 id=21309
/rrowtqGVAjmU2IRgrtEtGmOcrnM.jpg
iteration=4160 id=32235
/2u8uQchpm5zUIwax4XdWqZF6DNY.jpg
iteration=4161 id=329540
iteration=4162 id=26388
/iQxvrnEITUiriVnnM3Vf8IFt7c0.jpg
iteration=4163 id=49020
/qUHU6ugvkeAslMGkMyJTgfmPtf6.jpg
iteration=4164 id=159037
/ApQWdkfgsv2QQ8pylnGWR7eWrLH.jpg
iteration=4165 id=12838
/8djD0EaDCWy7JkSjjKezi1Udb

/y0j2UIuaoDybhoSU0j9MFnMZVqE.jpg
iteration=4312 id=43884
/8MS7d55axiPpSylyec1wBMVhpeO.jpg
iteration=4313 id=107315
None
iteration=4314 id=137955
None
iteration=4315 id=13064
/kvmeihN8VNABa0Lk19mEqAwtkMC.jpg
iteration=4316 id=59917
/7ZEzmTYcdj135aZ00SQNQWrWkyE.jpg
iteration=4317 id=157422
/5e1D0tx6B5ox5jjXWfgQH2Y1SG2.jpg
iteration=4318 id=310933
None
iteration=4319 id=7553
iteration=4320 id=12877
/7Bd0uPUbu7LiOrUHyQyp1pk93Qz.jpg
iteration=4321 id=39895
/389kslZbBbxAGhbB8jxAs1JxGAq.jpg
iteration=4322 id=102840
None
iteration=4323 id=55616
/sEPU1j7LtVnGyP0EZcQ03YLTpUM.jpg
iteration=4324 id=29697
/AmFI0bO30mOxdqmPrD0swnSxldI.jpg
iteration=4325 id=50037
/fwdmxO4eWWW6gD0F0hf9m7pVnrb.jpg
iteration=4326 id=94329
iteration=4327 id=53502
None
iteration=4328 id=202604
None
iteration=4329 id=289
/tfNuePdHrP9fp6K2VTJwHKKJf8C.jpg
iteration=4330 id=27374
/vSQX9RW1cYf7pTLURGqfUW6U3Q1.jpg
iteration=4331 id=26815
/bQk9Z0cj9OXB56sESStXVQgGKHC.jpg
iteration=4332 id=811
/9WmPbUHvyLc7zs3KdbopoRYBP7r.jpg
ite

None
iteration=4484 id=26837
/wE87f2Ls5te6BJcxGkZPFEmfiEu.jpg
iteration=4485 id=7859
/2WjZY5OcKQqrvnWSvfbiShfuVe5.jpg
iteration=4486 id=302579
None
iteration=4487 id=51955
None
iteration=4488 id=376004
/8q6AJYqJzZPrNSbttcwslXx7QgO.jpg
iteration=4489 id=158752
/yscOK6uHlORiUadSsixg80c2AT.jpg
iteration=4490 id=40658
None
iteration=4491 id=296943
None
iteration=4492 id=118612
/EJIstDMBj70my4IZVMX7uJlJ85.jpg
iteration=4493 id=138976
None
iteration=4494 id=323967
/nvl7kBSFn5XfOIE2CoUmoxdDBDh.jpg
iteration=4495 id=3080
/tnPgH5j3rPRajvF1iUqLH9WgWqz.jpg
iteration=4496 id=2667
iteration=4497 id=9459
/50tOx2lJXuszVjwCjNYHEXnPMWx.jpg
iteration=4498 id=11598
/pp67aWHcd1BzpLV8upRfgmwpt6s.jpg
iteration=4499 id=26916
None
iteration=4500 id=181940
None
iteration=4501 id=125263
None
iteration=4502 id=263503
None
iteration=4503 id=324322
/cjhGOLFJF1N5e3CjDJbyuTtC0m7.jpg
iteration=4504 id=331493
/exRCs4v31yh5CENBUo45g4fa9eX.jpg
iteration=4505 id=375950
/uu0MkrM0lw0v3pXnVqgD3UsF9pl.jpg
iteration=4506 id=2

/8MV1pFeOEp1zxlTjIGjtyIAfBda.jpg
iteration=4668 id=70478
/shR5YuxS5xz12IeRUywY06bm8X9.jpg
iteration=4669 id=35073
/zB3W6iCaYZSJtZSk0v6f8V8haA4.jpg
iteration=4670 id=9659
iteration=4671 id=10218
/yUTuHb75UKnKiUTpDn1kzUl7cd3.jpg
iteration=4672 id=391
/9gMkOYzk7P9QXwndhLRAC8cMZcJ.jpg
iteration=4673 id=43595
/mzSpJ0ne8YiGLYd3UIssneTV9mJ.jpg
iteration=4674 id=194588
None
iteration=4675 id=54897
/7oJePkds4pouWdkpQn5hYyfAcHk.jpg
iteration=4676 id=83588
/nPlv6si1wBGHh3Oclm6d2EuYCTa.jpg
iteration=4677 id=53256
/7KzhgodLSMjgZD0IcQrevNMovhz.jpg
iteration=4678 id=40920
None
iteration=4679 id=287815
None
iteration=4680 id=54702
iteration=4681 id=176074
None
iteration=4682 id=69270
/aw2FTZjiwEmI6MjcLyjfKUhQpgP.jpg
iteration=4683 id=5759
/k5uh8veAdTC7VLkcNsFerWFGedh.jpg
iteration=4684 id=402515
/ox8iMFZowBbS3QoOc8IcfOoWS71.jpg
iteration=4685 id=126141
None
iteration=4686 id=48035
/wa3yoalp8yxEwOMEkns0Jid86rr.jpg
iteration=4687 id=14758
/cPlNdSZeVPqgvea2qt8E5m8Wbm7.jpg
iteration=4688 id=13363
iteratio

In [74]:
len(poster)

4803

In [81]:
data['poster']='https://image.tmdb.org/t/p/w500'+pd.Series(poster)

In [82]:
data['poster']

0       https://image.tmdb.org/t/p/w500/gC3tW9a45RGOzz...
1       https://image.tmdb.org/t/p/w500/zRBaZxS5YauLvR...
2       https://image.tmdb.org/t/p/w500/HORpg5CSkmeQlA...
3       https://image.tmdb.org/t/p/w500/jyJ0aWWJamLUzE...
4       https://image.tmdb.org/t/p/w500/ubNevniKd5NYo6...
                              ...                        
4798    https://image.tmdb.org/t/p/w500/qmPhILSFQKaRyc...
4799    https://image.tmdb.org/t/p/w500/ek5Yac53PCliZV...
4800    https://image.tmdb.org/t/p/w500/qIK6rxMKMMYqkR...
4801    https://image.tmdb.org/t/p/w500/y8CGiAYo2E6Ld8...
4802    https://image.tmdb.org/t/p/w500/x41xFrJvmD1mlE...
Name: poster, Length: 4803, dtype: object

In [86]:
data.to_csv('movie_data.csv')

In [91]:
data['runtime'].isnull().sum()

0

In [122]:
list(data['original_title'].values)

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [123]:
json_string = json.dumps(list(data['original_title'].values))

In [124]:
with open('data.txt', 'w') as outfile:
    json.dump(json_string, outfile)

In [ ]:
pickle.dump(sig, open("model", 'wb'), protocol=4)